In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as scipy



In [11]:
#Study Data Files
happiness_data_path = "Resources/2018.csv"
alcohol_data_path = "Resources/Alcohol_Data.csv"

#Read Happiness and Alcohol Data
happiness_data_df = pd.read_csv(happiness_data_path)
alcohol_data_df = pd.read_csv(alcohol_data_path, skiprows = 4)

In [12]:
happiness_data_df.head()

,Overall rank,Country or region,Score,GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption
0,1,Finland,7.632,1.305,1.592,0.874,0.681,0.202,0.393
1,2,Norway,7.594,1.456,1.582,0.861,0.686,0.286,0.340
2,3,Denmark,7.555,1.351,1.590,0.868,0.683,0.284,0.408
3,4,Iceland,7.495,1.343,1.644,0.914,0.677,0.353,0.138
4,5,Switzerland,7.487,1.420,1.549,0.927,0.660,0.256,0.357


In [13]:
alcohol_data_df = alcohol_data_df["Country Name", "Country Code", "2018"]
alcohol_data_df

KeyError: ('Country Name', 'Country Code', '2018')